# Autoencoders - Aula Prática

## Configurações

Importando módulos necessários

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms.v2 as T

from torchsummary import summary

import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from math import ceil, sqrt
from tqdm.notebook import tqdm

## Introdução

Autoencoders são modelos que aprendem a representação de dados de entrada. Eles são compostos por duas partes: um encoder e um decoder. O encoder é responsável por mapear a entrada para um espaço latente, enquanto o decoder é responsável por mapear o espaço latente para a saída. A ideia é que o espaço latente seja uma representação mais compacta e significativa dos dados de entrada.

Nesta aula prática, vamos implementar um autoencoder para reconstruir imagens do dataset MNIST e entender como esse tipo de modelo funciona

## Carregando o dataset

Vamos começar carregando o dataset MNIST. O dataset é composto por imagens de dígitos escritos à mão, com dimensões 28x28 pixels.

In [2]:
mnist_path = '/pgeoprj2/ciag2024/dados/'

In [3]:
def plot_dataset(dataset, n=16, figsize=(6, 6)):
    cols = ceil(sqrt(n))
    rows = ceil(n / cols)
    fig, axes = plt.subplots(rows, cols, figsize=figsize)

    for i, ax in enumerate(axes.flat):
        if i == 0:
            print(f"image shape: {dataset[i][0].shape}")
        image, _, label = dataset[i]
        ax.imshow(image.squeeze(), cmap="gray")  # Squeeze removes the channel dimension
        ax.set_title(f"Label: {label}")
        ax.axis("off")

    plt.tight_layout()
    plt.show()

In [4]:
"""
Como queremos que o autoencoder aprenda a reconstruir a imagem de entrada,
reformulamos o dataset MNIST para que ele retorne a mesma imagem como entrada e saída.
"""
class AutoencoderMNIST(torch.utils.data.Dataset):
    def __init__(self, root, train=True, transform=None, input_transform=None, target_transform=None, download=False, return_labels=False):
        self.dataset = datasets.MNIST(root=root, train=train, transform=transform, download=download)
        self.input_transform = input_transform
        self.target_transform = target_transform
        self.return_labels = return_labels

    def __getitem__(self, idx):
        x, label = self.dataset[idx]
        y = x.clone()

        if self.input_transform:
            x = self.input_transform(x)
        if self.target_transform:
            y = self.target_transform(y)

        return (x, y, label) if self.return_labels else (x, y)
    
    def start_return_labels(self):
        self.return_labels = True
    
    def stop_return_labels(self):
        self.return_labels = False

    def __len__(self):
        return len(self.dataset)

In [5]:
mnist_train_ds = AutoencoderMNIST(
    mnist_path,
    train=True,
    transform=T.Compose([
        T.ToImage(),
        T.ToDtype(torch.float32, scale=True),
    ]),
    download=False,
    return_labels=True
)
mnist_test_ds = AutoencoderMNIST(
    mnist_path,
    train=False,
    transform=T.Compose([
        T.ToImage(),
        T.ToDtype(torch.float32, scale=True),
    ]),
)

print(
    "Tamanho do dataset de (treino, teste):", (len(mnist_train_ds), len(mnist_test_ds))
)

Tamanho do dataset de (treino, teste): (60000, 10000)


## Implementando o autoencoder

Vamos implementar um autoencoder convolucional simples, com 3 camadas convolucionais no encoder e 3 camadas convolucionais no decoder. A arquitetura do autoencoder é a seguinte:

- Encoder:
    - Conv2D com 16 filtros, kernel de tamanho 3x3, função de ativação ReLU
    - MaxPooling2D com pool size 2x2
    - Conv2D com 8 filtros, kernel de tamanho 3x3, função de ativação ReLU
    - MaxPooling2D com pool size 2x2
    - Conv2D com 2 filtros, kernel de tamanho 3x3, função de ativação ReLU
    - Camada Linear que mapeia os features maps para o espaço latente

- Decoder:
    - Camada Linear que mapeia o espaço latente para os features maps
    - Conv2D com 128 filtros, kernel de tamanho 3x3, função de ativação ReLU
    - UpSampling2D com size 2x2
    - Conv2D com 64 filtros, kernel de tamanho 3x3, função de ativação ReLU
    - UpSampling2D com size 2x2
    - Conv2D com 32 filtros, kernel de tamanho 3x3, função de ativação ReLU
    - Conv2D com 1 filtro, kernel de tamanho 3x3, função de ativação sigmoid

O tamanho do espaço latente é 2 para que possamos visualizar a representação dos dados no espaço latente.

In [6]:
class ConvolutionalEncoder(nn.Module):
    def __init__(self, latent_size=2, n_channels=3):
        super(ConvolutionalEncoder, self).__init__()
        self.latent_size = latent_size

        # reduzimos o tamanho da imagem
        # considerando input (n_channels, 28, 28)
        self.conv_layers = nn.Sequential(
            nn.Conv2d(n_channels, 16, 3, padding=1), # (16, 28, 28)
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2), # (16, 14, 14)
            nn.Conv2d(16, 8, 3, padding=1), # (8, 14, 14)
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),  # (8, 7, 7)
            nn.Conv2d(8, 2, 3, padding=1), # (2, 7, 7)
            nn.ReLU(),
        )

        # extraimos as features da imagem reduzida
        self.flatten = nn.Flatten() # (2 * 7 * 7,) = (98,)

        # finalmente fazemos o encoding da nossa imagem em um vetor n dimensional
        self.encoder = nn.Linear(98, self.latent_size)

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.encoder(x)
        return x

In [7]:
class ConvolutionalDecoder(nn.Module):
    def __init__(self, latent_size=2, n_channels=3):
        super(ConvolutionalDecoder, self).__init__()
        self.latent_size = latent_size

        # decodificamos a representação latente para uma imagem 7x7
        self.decoder = nn.Linear(self.latent_size, 98)
        self.unflatten = nn.Unflatten(1, (2, 7, 7)) # (2, 7, 7)

        # voltamos a imagem ao tamanho original
        self.conv_layers = nn.Sequential(
            nn.Conv2d(2, 8, 3, padding=1), # (8, 7, 7)
            nn.ReLU(),
            nn.Upsample(scale_factor=2), # (8, 14, 14)
            nn.Conv2d(8, 16, 3, padding=1), # (16, 14, 14)
            nn.ReLU(),
            nn.Upsample(scale_factor=2), # (16, 28, 28)
            nn.Conv2d(16, n_channels, 3, padding=1), # (n_channels, 28, 28)
            nn.Sigmoid(), # sigmoid na última camada para que o output esteja no intervalo [0, 1]
        )

    def forward(self, x):
        x = self.decoder(x)
        x = self.unflatten(x)
        x = self.conv_layers(x)
        return x

In [8]:
class Autoencoder(nn.Module):
    def __init__(self, latent_size=2, n_channels=3):
        super(Autoencoder, self).__init__()
        self.encoder = ConvolutionalEncoder(latent_size, n_channels)
        self.decoder = ConvolutionalDecoder(latent_size, n_channels)

    def forward(self, x, return_latent=False):
        latent = self.encoder(x)
        x = self.decoder(latent)

        return (x, latent) if return_latent else x

In [9]:
autoencoder = Autoencoder(latent_size=2, n_channels=1)
autoencoder.cuda()
summary(autoencoder, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
              ReLU-2           [-1, 16, 28, 28]               0
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4            [-1, 8, 14, 14]           1,160
              ReLU-5            [-1, 8, 14, 14]               0
         MaxPool2d-6              [-1, 8, 7, 7]               0
            Conv2d-7              [-1, 2, 7, 7]             146
              ReLU-8              [-1, 2, 7, 7]               0
           Flatten-9                   [-1, 98]               0
           Linear-10                    [-1, 2]             198
ConvolutionalEncoder-11                    [-1, 2]               0
           Linear-12                   [-1, 98]             294
        Unflatten-13              [-1, 2, 7, 7]               0
           Conv2d-14              [-

## Treinando o autoencoder

Vamos treinar o autoencoder para reconstruir imagens do dataset MNIST. O autoencoder será treinado por 20 épocas, com batch size 128 e otimizador Adam.

In [10]:
def train(model, optimizer, loss_fn, train_loader, val_loader=None, device='cpu', epochs=32):
    model.to(device)

    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0

        for i, (inputs, targets) in (pbar := tqdm(enumerate(train_loader), total=len(train_loader), unit='batch')):
            inputs, targets = inputs.to(device), targets.to(device)

            # Zero the gradient buffers
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            # A loss é computada contra o input pois queremos reconstruir a imagem de entrada
            loss = loss_fn(outputs, targets)
            loss /= inputs.size(0)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            pbar.set_description(f"Epoch {epoch + 1}/{epochs} - Loss: {train_loss/(i+1):.4f}")

        train_loss /= len(train_loader)

        # Validation phase
        if val_loader is not None:
            model.eval()
            val_loss = 0.0

            with torch.no_grad():
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)

                    outputs = model(inputs)
                    loss = loss_fn(outputs, targets)
                    loss /= inputs.size(0)

                    val_loss += loss.item()

            val_loss /= len(val_loader)

            print(f"Epoch {epoch + 1}/{epochs}: Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
        else:
            print(f"Epoch {epoch + 1}/{epochs}: Train Loss: {train_loss:.4f}")

In [11]:
mnist_train_ds.stop_return_labels()
mnist_test_ds.stop_return_labels()
train_dl = DataLoader(mnist_train_ds, batch_size=128, shuffle=True)
test_dl = DataLoader(mnist_test_ds, batch_size=128)

In [12]:
autoencoder = Autoencoder(latent_size=2, n_channels=1)

In [ ]:
epochs = 20
criterion = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train(autoencoder, optimizer, criterion, train_dl, test_dl, device=device, epochs=epochs)

  0%|          | 0/469 [00:00<?, ?batch/s]

Epoch 1/20: Train Loss: 90.5521 | Val Loss: 89.6147


  0%|          | 0/469 [00:00<?, ?batch/s]

Epoch 2/20: Train Loss: 87.8119 | Val Loss: 89.6146


  0%|          | 0/469 [00:00<?, ?batch/s]

...

  0%|          | 0/469 [00:00<?, ?batch/s]

Epoch 17/20: Train Loss: 87.8116 | Val Loss: 89.6146


  0%|          | 0/469 [00:00<?, ?batch/s]

Epoch 18/20: Train Loss: 87.8124 | Val Loss: 89.6146


  0%|          | 0/469 [00:00<?, ?batch/s]

## Analisando os resultados

Nessa etapa, vamos analisar os resultados obtidos pelo autoencoder. Vamos visualizar as imagens de entrada e as imagens reconstruídas pelo autoencoder. Além disso, vamos visualizar a representação dos dados no espaço latente.

In [ ]:
def plot_reconstructions(model, dataset, n=8, device='cpu', figsize=(12, 6), plot_label=False):
    fig, axes = plt.subplots(3 if plot_label else 2, n, figsize=figsize)
    images = torch.cat([dataset[i][0].unsqueeze(0) for i in range(n)], dim=0).to(device)
    labels = torch.cat([dataset[i][1].unsqueeze(0) for i in range(n)], dim=0)

    model.to(device)
    model.eval()
    with torch.no_grad():
        reconstructions = model(images)

    for i, (ax, im, rec, lbl) in enumerate(zip(range(n), images, reconstructions, labels)):
        im = im.cpu().squeeze()
        rec = rec.cpu().squeeze()
        lbl = lbl.cpu().squeeze()

        if plot_label:
            axes[0][ax].imshow(lbl, cmap="gray")
            axes[0][ax].axis("off")
            axes[1][ax].imshow(im, cmap="gray")
            axes[1][ax].axis("off")
            axes[2][ax].imshow(rec, cmap="gray")
            axes[2][ax].axis("off")
        else:
            axes[0][ax].imshow(im, cmap="gray")
            axes[0][ax].axis("off")
            axes[1][ax].imshow(rec, cmap="gray")
            axes[1][ax].axis("off")

        if i == n//2:
            if plot_label:
                axes[0][ax].set_title("Original")
                axes[1][ax].set_title("Sample")
                axes[2][ax].set_title("Reconstruction")
            else:
                axes[0][ax].set_title("Sample")
                axes[1][ax].set_title("Reconstruction")

    plt.tight_layout()
    plt.show()

In [ ]:
plot_reconstructions(autoencoder, mnist_test_ds, n=9, device=device, figsize=(12, 6))

In [ ]:


def plot_latent_space(encoder, dataset, n_classes=10, batch_size=128, device='cpu', reduce_dims=None):
    # Get the latent representations for all test data
    latent_representations = []
    labels_agg = []
    classes = list(range(n_classes))

    dataset.start_return_labels()
    test_dl_with_labels = DataLoader(dataset, batch_size=batch_size)

    encoder.to(device)
    encoder.eval()
    with torch.no_grad():
        for images, _, labels in test_dl_with_labels:
            images = images.to(device)
            latent_vectors = encoder(images)
            latent_representations.append(latent_vectors.cpu().numpy())
            labels_agg.append(labels.numpy())

    # Concatenate all latent representations
    latent_representations = np.concatenate(latent_representations, axis=0)
    labels_agg = np.concatenate(labels_agg, axis=0)
    
    if reduce_dims is not None:
        if reduce_dims == 'pca':
            pca = PCA(n_components=2, random_state=42)
            latent_representations = pca.fit_transform(latent_representations)
        elif reduce_dims == 'tsne':
            tsne = TSNE(n_components=2, random_state=42)
            latent_representations = tsne.fit_transform(latent_representations)

    # Plot the latent representations
    plt.figure(figsize=(10, 8))
    cmap = plt.get_cmap('jet', 10)
    for c in classes:
        indices = np.where(labels_agg == c)
        plt.scatter(
            latent_representations[indices, 0],
            latent_representations[indices, 1],
            s=3,
            label=f"{c}",
            c=[cmap(c)],
            alpha=0.5,
        )
    # add the labels mapping
    plt.legend(markerscale=5)
    plt.title('Latent Representations of Test Data')
    plt.xlabel('Latent Dimension 1')
    plt.ylabel('Latent Dimension 2')
    plt.grid(True)
    plt.show()

In [ ]:
plot_latent_space(autoencoder.encoder, mnist_test_ds, reduce_dims=None, device=device)

Observe que os digitos estão organizados no espaço latente de acordo com suas características. Por exemplo, os digitos 0 e 1 estão distantes, enquanto os digitos 3 e 8 estão juntos. Isso mostra que o autoencoder aprendeu a representação dos dados de forma significativa.

# Exercícios

## Exercício 1 - Denoising Autoencoder

Tendo em vista que o autoencoder é capaz de aprender a representação dos dados, podemos utilizá-lo para remover ruídos das imagens. Implemente um denoising autoencoder, que recebe imagens com ruídos como entrada e tenta reconstruir a imagem original.

<details>
<summary>Dica:</summary>
Você só vai precisar de mexer no dataset de entrada, adicionando ruídos às imagens.
</details>

In [ ]:
mnist_train_ds = AutoencoderMNIST(
    mnist_path,
    train=True,
    transform=T.Compose([
        T.ToImage(),
        T.ToDtype(torch.float32, scale=True),
    ]),
    input_transform=T.GaussianNoise(0, 0.1),
    return_labels=True,
    download=True,
)
mnist_test_ds = AutoencoderMNIST(
    mnist_path,
    train=False,
    transform=T.Compose([
        T.ToImage(),
        T.ToDtype(torch.float32, scale=True),
    ]),
    input_transform=T.GaussianNoise(0, 0.1)
)

print(
    "Tamanho do dataset de (treino, teste):", (len(mnist_train_ds), len(mnist_test_ds))
)

In [ ]:
plot_dataset(mnist_train_ds, 20)

In [ ]:
denoising_autoencoder = Autoencoder(latent_size=8, n_channels=1)

In [ ]:
epochs = 20

mnist_train_ds.stop_return_labels()
mnist_test_ds.stop_return_labels()
train_dl = DataLoader(mnist_train_ds, batch_size=128, shuffle=True)
test_dl = DataLoader(mnist_test_ds, batch_size=128)

criterion = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(denoising_autoencoder.parameters(), lr=1e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train(denoising_autoencoder, optimizer, criterion, train_dl, test_dl, device=device, epochs=epochs)

In [ ]:
plot_reconstructions(denoising_autoencoder, mnist_test_ds, n=9, device=device, plot_label=True)

In [ ]:
plot_latent_space(denoising_autoencoder.encoder, mnist_test_ds, reduce_dims=None, device=device)

## Exercício 2 - Removendo marca d'água

Implemente um autoencoder que remove a marca d'água de imagens. Para isso, adicione uma marca d'água a algumas imagens do dataset MNIST e treine o autoencoder para removê-la.

In [ ]:
class Watermark(object):
    def __init__(self, watermark, clamp=True):
        self.watermark = watermark
        self.w_width, self.w_height = watermark.shape[-2:]
        self.clamp = clamp

    def __call__(self, image):
        width, height = image.shape[-2:]
        x_offset = torch.randint(0, width - self.w_width, (1,)).item()
        y_offset = torch.randint(0, height - self.w_height, (1,)).item()
        im_patch = image[..., x_offset:x_offset+self.w_width, y_offset:y_offset+self.w_height]
        im_patch += self.watermark.unsqueeze(0)
        if self.clamp:
            image = torch.clamp(image, 0, 1)
        return image

ciag_watermark = [
    [1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1],
    [1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1],
]

In [ ]:
mnist_train_ds = AutoencoderMNIST(
    mnist_path,
    train=True,
    transform=T.Compose([
        T.ToImage(),
        T.ToDtype(torch.float32, scale=True),
    ]),
    input_transform=Watermark(torch.tensor(ciag_watermark)),
    return_labels=True,
    download=True,
)
mnist_test_ds = AutoencoderMNIST(
    mnist_path,
    train=False,
    transform=T.Compose([
        T.ToImage(),
        T.ToDtype(torch.float32, scale=True),
    ]),
    input_transform=Watermark(torch.tensor(ciag_watermark)),
)

print(
    "Tamanho do dataset de (treino, teste):", (len(mnist_train_ds), len(mnist_test_ds))
)

In [ ]:
plot_dataset(mnist_train_ds, 20)

In [ ]:
watermark_autoencoder = Autoencoder(latent_size=4, n_channels=1)

In [ ]:
epochs = 20

mnist_train_ds.stop_return_labels()
mnist_test_ds.stop_return_labels()
train_dl = DataLoader(mnist_train_ds, batch_size=128, shuffle=True)
test_dl = DataLoader(mnist_test_ds, batch_size=128)

criterion = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(watermark_autoencoder.parameters(), lr=1e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train(watermark_autoencoder, optimizer, criterion, train_dl, test_dl, device=device, epochs=epochs)

In [ ]:
plot_reconstructions(watermark_autoencoder, mnist_test_ds, n=8, device=device, plot_label=True)

In [ ]:
plot_latent_space(watermark_autoencoder.encoder, mnist_test_ds, reduce_dims='pca', device=device)

## Exercício 3 - Autoencoder com regularização

Implemente um autoencoder com regularização no espaço latente. Adicione uma regularização L1 ou L2 no espaço latente e treine o autoencoder.

<details>
<summary>Dica:</summary>
Altere a função de treino para aceitar mais um parâmetro, que será a regularização no espaço latente.
</details>

In [ ]:
mnist_train_ds = AutoencoderMNIST(
    mnist_path,
    train=True,
    transform=T.Compose([
        T.ToImage(),
        T.ToDtype(torch.float32, scale=True),
    ]),
    download=True,
    return_labels=True
)
mnist_test_ds = AutoencoderMNIST(
    mnist_path,
    train=False,
    transform=T.Compose([
        T.ToImage(),
        T.ToDtype(torch.float32, scale=True),
    ])
)

plot_dataset(mnist_train_ds, 20)

In [ ]:
def train_reg(model, optimizer, loss_fn, reg_loss_fn, train_loader, val_loader=None, device='cpu', epochs=32):
    model.to(device)

    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_reconstruction_loss = 0.0
        train_regularization_loss = 0.0

        for i, (inputs, targets) in (pbar := tqdm(enumerate(train_loader), total=len(train_loader), unit='batch')):
            inputs, targets = inputs.to(device), targets.to(device)

            # Zero the gradient buffers
            optimizer.zero_grad()

            # Forward pass
            outputs, latent = model(inputs, return_latent=True)
            # A loss é computada contra o input pois queremos reconstruir a imagem de entrada
            reconstruction_loss = loss_fn(outputs, targets)
            reconstruction_loss /= inputs.size(0)
            train_reconstruction_loss += reconstruction_loss.item()

            regularization_loss = reg_loss_fn(latent)
            regularization_loss /= inputs.size(0)
            train_regularization_loss += regularization_loss.item()

            loss = reconstruction_loss + regularization_loss

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            pbar.set_description(f"Epoch {epoch + 1}/{epochs} - Loss: {train_loss/(i+1):.4f}")

        train_loss /= len(train_loader)
        train_reconstruction_loss /= len(train_loader)
        train_regularization_loss /= len(train_loader)

        # Validation phase
        if val_loader is not None:
            model.eval()
            val_loss = 0.0
            val_reconstruction_loss = 0.0
            val_regularization_loss = 0.0

            with torch.no_grad():
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)

                    outputs, latent = model(inputs, return_latent=True)
                    reconstruction_loss = loss_fn(outputs, targets)
                    reconstruction_loss /= inputs.size(0)
                    val_reconstruction_loss += reconstruction_loss.item()

                    regularization_loss = reg_loss_fn(latent)
                    regularization_loss /= inputs.size(0)
                    val_regularization_loss += regularization_loss.item()

                    loss = reconstruction_loss + regularization_loss

                    val_loss += loss.item()

            val_loss /= len(val_loader)
            val_reconstruction_loss /= len(val_loader)
            val_regularization_loss /= len(val_loader)

            print(f"Epoch {epoch + 1}/{epochs}: Train Loss: {train_loss:.4f} Train Reconstruction Loss: {train_reconstruction_loss:.4f} \
Train Regularization Loss: {train_regularization_loss} | Val Loss: {val_loss:.4f} Val Reconstruction Loss: {val_reconstruction_loss:.4f} \
Val Regularization Loss: {val_regularization_loss}")
        else:
            print(f"Epoch {epoch + 1}/{epochs}: Train Loss: {train_loss:.4f} Train Reconstruction Loss: {train_reconstruction_loss:.4f} \
Train Regularization Loss: {train_regularization_loss}")

In [ ]:
regularized_autoencoder = Autoencoder(latent_size=2, n_channels=1)

In [ ]:
epochs = 20
lamda = 1

mnist_train_ds.stop_return_labels()
mnist_test_ds.stop_return_labels()
train_dl = DataLoader(mnist_train_ds, batch_size=128, shuffle=True)
test_dl = DataLoader(mnist_test_ds, batch_size=128)

criterion = nn.MSELoss(reduction='sum')
l2_loss = nn.MSELoss(reduction='sum')
regularization_loss = lambda x: lamda * l2_loss(x, torch.zeros_like(x))

l1_loss = nn.L1Loss(reduction='sum')
regularization_loss = lambda x: lamda * l1_loss(x, torch.zeros_like(x))

optimizer = torch.optim.Adam(regularized_autoencoder.parameters(), lr=1e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_reg(regularized_autoencoder, optimizer, criterion, regularization_loss, train_dl, test_dl, device=device, epochs=epochs)

In [ ]:
plot_reconstructions(regularized_autoencoder, mnist_test_ds, n=8, device=device)

In [ ]:
plot_latent_space(regularized_autoencoder.encoder, mnist_test_ds, reduce_dims=None, device=device)